In [ ]:
import numpy as np
from cirq import (
    CNOT,
    I,
    X,
    Y,
    Z,
    LineQubit,
    Circuit,
    ops,
    unitary,
)
from mitiq import Executor, Observable, PauliString
from mitiq.interface.mitiq_cirq import compute_density_matrix
from mitiq.cdr._testing import random_x_z_cnot_circuit
from mitiq.pec.representations.learning import learn_biased_noise_parameters

Test with randomly generated circuit, compiled into (X, Z, CNOT) for `cdr.generate_training_circuits`

In [ ]:
circuit = random_x_z_cnot_circuit(
    LineQubit.range(3), n_moments=5, random_state=1
)
print(circuit)

Test with some example parameters (small offset to avoid long execution time)

In [ ]:
epsilon = 0.7
eta = 1000
gate = CNOT
offset = -0.01
observable = Observable(PauliString("XZ"), PauliString("YY"))

# Define biased noise channel
a = 1 - epsilon
b = epsilon * (3 * eta + 1) / (3 * (eta + 1))
c = epsilon / (3 * (eta + 1))

mix = [
    (a, unitary(I)),
    (b, unitary(Z)),
    (c, unitary(X)),
    (c, unitary(Y)),
]

print(circuit)
print(circuit.with_noise(ops.MixedUnitaryChannel(mix)))

In [ ]:
def ideal_executor(circ: Circuit) -> np.ndarray:
    return compute_density_matrix(circ, noise_level=(0.0,))

def noisy_executor(circ: Circuit) -> np.ndarray:
    noisy_circ = circ.with_noise(ops.MixedUnitaryChannel(mix))
    return ideal_executor(noisy_circ)

In [ ]:

ideal_executor = Executor(ideal_executor)
noisy_executor = Executor(noisy_executor)

from mitiq.cdr import generate_training_circuits
epsilon0 = (1 + offset) * epsilon
eta0 = (1 + offset) * eta
num_training_circuits = 10
training_circuits = generate_training_circuits(
        circuit=circuit,
        num_training_circuits=num_training_circuits,
        fraction_non_clifford=0,
        method_select="uniform",
        method_replace="closest",
    )

ideal_values = np.array(
        [ideal_executor.evaluate(t, observable) for t in training_circuits]
    )


x0 = np.array(
        [epsilon0, eta0]
    )  # initial parameter values for optimization

print(ideal_values)

print(x0)


In [ ]:
from mitiq.pec import execute_with_pec
from mitiq.pec.representations.biased_noise import (
    represent_operation_with_local_biased_noise,
)
epsilon = x0[0]
eta = x0[1]
operation = gate
qreg = LineQubit.range(operation.num_qubits())
representations = [
        represent_operation_with_local_biased_noise(
            Circuit(operation.on(*qreg)),
            epsilon,
            eta,
        )
    ]

mitigated = execute_with_pec(
        circuit=circuit,
        observable=observable,
        executor=noisy_executor,
        representations=representations,
    )


np.sum((mitigated * np.ones(num_training_circuits) - ideal_values)
            ** 2
        )/ num_training_circuits

In [ ]:
[epsilon_opt, eta_opt] = learn_biased_noise_parameters(
    operation=gate,
    circuit=circuit,
    ideal_executor=Executor(ideal_executor),
    noisy_executor=Executor(noisy_executor),
    num_training_circuits=10,
    epsilon0=(1 + offset) * epsilon,
    eta0=(1 + offset) * eta,
    observable=observable
)

print(epsilon-epsilon_opt)
print(eta-eta_opt)